In [1]:
pip install mlxtend


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [52]:
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sns  
import sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder  
from sklearn.neighbors import NearestNeighbors                  
from sklearn.cluster import KMeans, DBSCAN                       
from sklearn.metrics import silhouette_score, davies_bouldin_score 
from sklearn.decomposition import PCA                            
import os               
from datetime import datetime 
import warnings         
warnings.filterwarnings('ignore')
import numpy as np


In [3]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [4]:
# Load Customers.csv
customers = pd.read_csv(r"C:\Users\yadav\Videos\Customers.csv")
products = pd.read_csv(r"C:\Users\yadav\Videos\Products.csv")
transactions = pd.read_csv(r"C:\Users\yadav\Videos\Transactions.csv")

In [5]:
customers.head(20)


,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15
5,C0006,Brittany Palmer,South America,2024-01-07
6,C0007,Paul Graves,Asia,2022-06-18
7,C0008,David Li,North America,2024-01-13
8,C0009,Joy Clark,Europe,2023-08-14
9,C0010,Aaron Cox,Europe,2022-12-15


In [6]:
products.head() 


,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [7]:
transactions.head(10)

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68
5,T00442,C0188,P067,2024-12-26 14:40:03,1,300.68,300.68
6,T00490,C0195,P067,2024-11-24 11:49:48,3,902.04,300.68
7,T00536,C0008,P067,2024-09-22 06:13:59,1,300.68,300.68
8,T00564,C0157,P067,2024-12-07 17:57:40,3,902.04,300.68
9,T00631,C0130,P067,2024-05-14 23:14:59,2,601.36,300.68


In [8]:
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])


In [9]:
# 1. Join Transactions with Customers on CustomerID
transactions_with_customer = pd.merge(transactions, customers, on='CustomerID', how='left')


In [10]:
final_df= pd.merge(transactions_with_customer, products, on='ProductID', how='left')


In [11]:
final_df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22,SoundWave Smartwatch,Electronics,459.86
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07,SoundWave Smartwatch,Electronics,459.86
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21,SoundWave Smartwatch,Electronics,459.86
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11,SoundWave Smartwatch,Electronics,459.86


# Task 2: Lookalike Model

In [39]:
customer_features_df = final_df.groupby('CustomerID').agg(
    Region=('Region', 'first'), # Customer Profile Feature: Region (assuming region doesn't change)
    SignupDate=('SignupDate', 'first'), # Customer Profile Feature: Signup Date
    TransactionFrequency=('TransactionID', 'count'), # Transaction History Feature: Frequency
    AvgTransactionValue=('TotalValue', 'mean'), # Transaction History Feature: Avg Monetary Value
    TotalTransactionValue=('TotalValue', 'sum') # Transaction History Feature: Total Monetary Value
)


In [40]:
# Calculate Days since signup as customer age (relative to a recent date in the dataset)
reference_date = final_df['TransactionDate'].max() # Using latest transaction date as reference
customer_features_df['CustomerAgeDays'] = (reference_date - customer_features_df['SignupDate']).dt.days
customer_features_df = customer_features_df.drop(columns=['SignupDate']) # No longer need SignupDate column


In [41]:
try:
    final_df['SignupDate'] = pd.to_datetime(final_df['SignupDate'])
    print("'SignupDate' column ensured to be in datetime format.")
except Exception as e:
    print(f"Error converting 'SignupDate' to datetime: {e}")
    print("Please check your 'SignupDate' column format and ensure it's convertible.")
    exit() # Stop execution if conversion is critical


final_df['SignupYear'] = final_df['SignupDate'].dt.year
final_df['SignupMonth_Num'] = final_df['SignupDate'].dt.month  # Month as number (1-12)
final_df['SignupDay'] = final_df['SignupDate'].dt.day


# Calculate Tenure (using a reference date, e.g., end of 2024 or today)
reference_date = pd.to_datetime('2024-12-31')  # Example: End of 2024
final_df['TenureDays'] = (reference_date - final_df['SignupDate']).dt.days


print("\n'SignupYear', 'SignupMonth_Num', 'SignupDay', and 'TenureDays' columns created.")
print(final_df[['SignupDate', 'SignupYear', 'SignupMonth_Num', 'SignupDay', 'TenureDays']].head()) # Display sample

'SignupDate' column ensured to be in datetime format.

'SignupYear', 'SignupMonth_Num', 'SignupDay', and 'TenureDays' columns created.
  SignupDate  SignupYear  SignupMonth_Num  SignupDay  TenureDays
0 2022-12-03        2022               12          3         759
1 2024-09-04        2024                9          4         118
2 2024-04-04        2024                4          4         271
3 2024-04-11        2024                4         11         264
4 2022-03-15        2022                3         15        1022


In [42]:
# One-hot encode Region
encoder = OneHotEncoder(sparse_output=False)
region_encoded = encoder.fit_transform(customer_features_df[['Region']])
region_df = pd.DataFrame(region_encoded,
                           columns=encoder.get_feature_names_out(['Region']),
                           index=customer_features_df.index)


In [43]:
customer_features_df = pd.concat([customer_features_df, region_df], axis=1).drop(columns=['Region'])


In [44]:
# Scale numerical features
numerical_features = ['TransactionFrequency', 'AvgTransactionValue', 'TotalTransactionValue']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features_df[numerical_features])
scaled_df = pd.DataFrame(scaled_features, columns=numerical_features, index=customer_features_df.index)

customer_features_scaled_df = pd.concat([customer_features_df.drop(columns=numerical_features), scaled_df], axis=1)


In [45]:
# Select features for similarity calculation
feature_cols = ['Region_Asia', 'Region_Europe', 'Region_North America', 'Region_South America',
                'TransactionFrequency', 'AvgTransactionValue', 'TotalTransactionValue']
customer_similarity_features = customer_features_scaled_df[feature_cols].fillna(0)


In [46]:
# Calculate cosine similarity matrix
customer_similarity_matrix = cosine_similarity(customer_similarity_features)
customer_similarity_df = pd.DataFrame(customer_similarity_matrix, index=customer_features_scaled_df.index, columns=customer_features_scaled_df.index)


In [47]:
lookalike_results = {}
customer_ids_to_evaluate = customers['CustomerID'].head(20).tolist()

for cust_id in customer_ids_to_evaluate:
    if cust_id in customer_similarity_df.columns:
        similarity_series = customer_similarity_df[cust_id].sort_values(ascending=False)
        top_similar_customers = similarity_series[1:4] # Exclude the customer themselves and take top 3
        lookalike_results[cust_id] = []
        for similar_cust_id, score in top_similar_customers.items():
            lookalike_results[cust_id].append({'cust_id': similar_cust_id, 'score': score})



In [48]:
# Format the output for Lookalike.csv
output_list = []
for cust_id, lookalikes in lookalike_results.items():
    lookalike_map = {}
    lookalike_map['cust_id'] = cust_id
    lookalike_map['lookalikes'] = [{'cust_id': item['cust_id'], 'score': item['score']} for item in lookalikes]
    output_list.append(lookalike_map)


In [49]:
output_df = pd.DataFrame(output_list)


In [50]:
def format_lookalikes_map(lookalikes_list):
    return {item['cust_id']: item['score'] for item in lookalikes_list}

output_df['lookalike_map'] = output_df['lookalikes'].apply(format_lookalikes_map)
output_df = output_df.drop(columns=['lookalikes'])
output_df_final = output_df.rename(columns={'cust_id': 'CustomerID'})
output_df_final.to_csv("Lookalike.csv", index=False)


print(output_df_final.head(20).to_markdown(index=False))


| CustomerID   | lookalike_map                                                                           |
|:-------------|:----------------------------------------------------------------------------------------|
| C0001        | {'C0137': 0.9997616475513842, 'C0152': 0.9995103764905026, 'C0107': 0.9641690971679806} |
| C0002        | {'C0043': 0.9789986810881014, 'C0142': 0.9758976896264445, 'C0088': 0.9544024281819797} |
| C0003        | {'C0133': 0.9870616987011798, 'C0052': 0.9754103448062151, 'C0112': 0.9415781873172182} |
| C0004        | {'C0108': 0.9827182639674003, 'C0113': 0.9785391786723805, 'C0165': 0.9738306048704476} |
| C0005        | {'C0159': 0.9993487455683338, 'C0123': 0.9825990842253818, 'C0186': 0.9717374531257307} |
| C0006        | {'C0168': 0.978264001321209, 'C0158': 0.971144620969119, 'C0171': 0.9387564178572096}   |
| C0007        | {'C0140': 0.9798101409546569, 'C0080': 0.946117626429296, 'C0078': 0.94567319179717}    |
| C0008        | {'C0139': 0.97219883

In [ ]:

lookalike_results = {}

for customer_id in customers['CustomerID'].iloc[:20]:
    lookalikes = get_lookalikes(customer_id, knn, customer_features_final)
    lookalike_results[customer_id] = lookalikes

lookalike_df = pd.DataFrame.from_dict(lookalike_results, orient='index').reset_index()
lookalike_df.columns = ['CustomerID', 'Lookalike1', 'Lookalike2', 'Lookalike3']

lookalike_df.to_csv('Lookalike.csv', index=False)